# CONSTANTS

In [1]:
import CONSTANTS as c

NR_OF_ENSEMBLED_MODELS = c.NR_OF_ENSEMBLED_MODELS

# LIBRARIES

In [2]:
import numpy as np

from imblearn.over_sampling import SMOTEN

# DATA SOURCE

In [3]:
X = np.load(r'C:\Users\yunus\Desktop\Feature Engineered Data\X.npy')
Y = np.load(r'C:\Users\yunus\Desktop\Feature Engineered Data\Y.npy')
X_TIME = np.load(r'C:\Users\yunus\Desktop\Feature Engineered Data\X_TIME.npy')

In [4]:
Y = Y[:,0,:,0]
X = X[:,:,:,0]

In [5]:
Y  = Y[:, int(Y.shape[1]/2):]

# ANALYSIS

## Bagging

In [6]:
aUniqueClasses =  np.unique(Y, axis = 0, return_counts=True)
iNrOfUniqueClasses = len(aUniqueClasses[0])

iSizeOfBag = int(len(Y)/NR_OF_ENSEMBLED_MODELS)
while iSizeOfBag%NR_OF_ENSEMBLED_MODELS != 0:
     iSizeOfBag =  iSizeOfBag + 1


iSizeOfEachClass = int(iSizeOfBag/NR_OF_ENSEMBLED_MODELS)
Y_bags = []
X_bags = []
X_TIME_bags = [] 
for i in range(NR_OF_ENSEMBLED_MODELS):
    Y_sampled = np.empty((0, Y.shape[1]))
    X_sampled = np.empty((0, X.shape[1], X.shape[2]))
    X_TIME_sampled = np.empty((0, X_TIME.shape[1]))
    for j in range(iNrOfUniqueClasses):
        aClassToSample =  aUniqueClasses[0][j]
        aIndicesToSample = np.where((Y == aClassToSample).all(axis=1))[0]
        
        iIndicesSampled = np.random.choice(aIndicesToSample, iSizeOfEachClass)
        Y_sampled = np.concatenate((Y_sampled, Y[iIndicesSampled]))
        X_sampled = np.concatenate((X_sampled, X[iIndicesSampled]))
        X_TIME_sampled = np.concatenate((X_TIME_sampled, X_TIME[iIndicesSampled]))
        
    Y_bags.append(Y_sampled)
    X_bags.append(X_sampled)
    X_TIME_bags.append(X_TIME_sampled)
    

Y_bags = np.array(Y_bags)
X_bags = np.array(X_bags)
X_TIME_bags = np.array(X_TIME_bags) 

## SMOTEN Oversampling

In [44]:
Y_ORDINAL = np.sum(Y,axis = 1)

X_RESHAPED = np.reshape(X,(X.shape[0], -1))
X_RESHAPED = np.concatenate((X_RESHAPED,X_TIME), axis = 1)

oSmote = SMOTEN(random_state=42)

X_res, y_res = oSmote.fit_resample(X_RESHAPED, Y_ORDINAL)

In [68]:
X_TIME_smoten = X_res[:, -X_TIME.shape[1]:]
X_smoten = X_res[:, :-X_TIME.shape[1]]
X_smoten = np.reshape(X_smoten, (-1, X.shape[1], X.shape[2]))
Y_smoten = np.zeros((y_res.shape[0], Y.shape[1]) )

aUniqueVals = np.unique(y_res, return_counts=False)
for iLabel in aUniqueVals:
    aIndicesToAdjust = np.where(y_res == iLabel)[0]
    Y_smoten[aIndicesToAdjust,:int(iLabel)] = 1

In [ ]:
# check quality of X_TIME_smoten

# SAVE

In [8]:
# BAGS
np.save(r'C:\Users\yunus\Desktop\Augmented Data\Bagging\Y_bags.npy', Y_bags)
np.save(r'C:\Users\yunus\Desktop\Augmented Data\Bagging\X_bags.npy', X_bags)
np.save(r'C:\Users\yunus\Desktop\Augmented Data\Bagging\X_TIME_bags.npy', X_TIME_bags)

In [69]:
# SMOTEN
np.save(r'C:\Users\yunus\Desktop\Augmented Data\Bagging\X_smoten.npy', X_smoten)
np.save(r'C:\Users\yunus\Desktop\Augmented Data\Bagging\Y_smoten.npy', Y_smoten)
np.save(r'C:\Users\yunus\Desktop\Augmented Data\Bagging\X_TIME_smoten.npy', X_TIME_smoten)